##### About this notebook:

In [ ]:
#-----------------------------------------------------------------------------------------------------------------------------
# Author:             Erick Rico Esparza
# Dates:              Jan 23 - 26, 2026
# Description:        A methodological sensitivity check to finalize event definition and temporal resolution
# Organization:       Tampere University / Institute of Atmospheric Sciences and Climate Change (ICAyCC-UNAM)
#-----------------------------------------------------------------------------------------------------------------------------

# Stage 1.2

## 1. Setup & constants

In [138]:
# Core scientific stack
import xarray as xr
import numpy as np
import pandas as pd

# Plotting
import matplotlib
matplotlib.use("Agg") # non-interactive backend for saving figures
import matplotlib.pyplot as plt
from matplotlib.colors import TwoSlopeNorm
import matplotlib.patches as mpatches
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import calendar
from IPython.display import display, HTML
from matplotlib.lines import Line2D

# Stats
from scipy.stats import ttest_ind

# Warnings
from xarray.coding.variables import SerializationWarning
import warnings
warnings.filterwarnings("ignore", category=SerializationWarning)

In [14]:
# I/O utilities
import os
import re
import zipfile
from pathlib import Path

In [3]:
# Display formatting
pd.options.display.float_format = '{:,.2f}'.format

In [4]:
# Domain & constants
LON_MIN, LON_MAX = -120, -85
LAT_MIN, LAT_MAX = 12, 33
LON_CDMX, LAT_CDMX = -99.13, 19.43

# MCMA box
SW_lat, SW_lon = 18.3, -100.9
NE_lat, NE_lon = 20.7, -97.4
MCMA_BOX = (SW_lon, SW_lat, NE_lon - SW_lon, NE_lat - SW_lat)

pollutants = ["PM2.5", "PM10"]

In [5]:
# Daily (city-mean) dataset period
DAILY_START_YEAR = 2012
DAILY_END_YEAR   = 2024

# Hourly RAMA datasets
HOURLY_PM10_START_YEAR = 1995
HOURLY_PM10_END_YEAR   = 2023

HOURLY_PM25_START_YEAR = 2003
HOURLY_PM25_END_YEAR   = 2023

## 2. Load reanalysis + pre-processing (NARR 500 hPa)

In [6]:
# 500 hPa
H500 = xr.open_dataset("hgt500_mex_2012_2024.nc")["hgt"]
U500 = xr.open_dataset("uwnd500_mex_2012_2024.nc")["uwnd"]
V500 = xr.open_dataset("vwnd500_mex_2012_2024.nc")["vwnd"]

# Ensure datetime
H500 = H500.assign_coords(time=pd.to_datetime(H500.time.values))
U500 = U500.assign_coords(time=pd.to_datetime(U500.time.values))
V500 = V500.assign_coords(time=pd.to_datetime(V500.time.values))

# Safety: align time axes
H500, U500, V500 = xr.align(H500, U500, V500, join="inner")

lon2d, lat2d = H500["lon"].values, H500["lat"].values

## 3. PM daily + monthly extremes (p90 and p10)

In [7]:
df = pd.read_csv("pm_cdmx_citymean_daily_2012_2024.csv")
df["DATE"] = pd.to_datetime(df["DATE"])
df = df.set_index("DATE").sort_index()

def extreme_events_by_month(series: pd.Series, q: float) -> dict:
    """
    Returns dict: month(int 1-12) -> DatetimeIndex of event days
    Extreme is defined within each month-year using quantile q.
      - q=0.90 -> high extremes (>= threshold)
      - q=0.10 -> low extremes  (<= threshold)
    """
    out = {m: [] for m in range(1, 13)}

    for period, s in series.groupby(series.index.to_period("M")):
        if len(s) == 0:
            continue

        thr = s.quantile(q)

        if q >= 0.5:
            ev = s[s >= thr].index
        else:
            ev = s[s <= thr].index

        m = period.month  # robust: month from the group itself
        out[m].extend(ev.tolist())

    return {m: pd.DatetimeIndex(sorted(set(out[m]))) for m in out}

events_p90 = {p: extreme_events_by_month(df[p].dropna(), q=0.90) for p in pollutants}
events_p10 = {p: extreme_events_by_month(df[p].dropna(), q=0.10) for p in pollutants}

print("Event day count by month (p90 within each month-year):")
for p in pollutants:
    print(p, {m: len(events_p90[p][m]) for m in range(1, 13)})

print("\nEvent day count by month (p10 within each month-year):")
for p in pollutants:
    print(p, {m: len(events_p10[p][m]) for m in range(1, 13)})

print("\nDaily dataset info:")
print(df.info())

Event day count by month (p90 within each month-year):
PM2.5 {1: 49, 2: 39, 3: 52, 4: 39, 5: 48, 6: 37, 7: 52, 8: 55, 9: 39, 10: 49, 11: 38, 12: 52}
PM10 {1: 49, 2: 38, 3: 52, 4: 40, 5: 48, 6: 37, 7: 51, 8: 51, 9: 37, 10: 49, 11: 39, 12: 51}

Event day count by month (p10 within each month-year):
PM2.5 {1: 50, 2: 38, 3: 51, 4: 40, 5: 48, 6: 37, 7: 54, 8: 53, 9: 37, 10: 51, 11: 38, 12: 53}
PM10 {1: 49, 2: 38, 3: 52, 4: 40, 5: 49, 6: 37, 7: 51, 8: 52, 9: 37, 10: 49, 11: 38, 12: 51}

Daily dataset info:
<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 4555 entries, 2012-01-01 to 2024-12-31
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   PM10    4555 non-null   float64
 1   PM2.5   4555 non-null   float64
dtypes: float64(2)
memory usage: 106.8 KB
None


## 4. Climatology + anomalies  (daily climatology smoothed with 31-day rolling mean)

In [8]:
def daily_climatology(da):
    da = da.assign_coords(time=pd.to_datetime(da.time.values))
    da_noleap = da.sel(time=~((da.time.dt.month==2) & (da.time.dt.day==29)))
    clim = da_noleap.groupby("time.dayofyear").mean("time")
    clim = clim.rolling(dayofyear=31, center=True, min_periods=1).mean()
    return clim

clim500 = daily_climatology(H500)

## 5. Monthly composites + 3×4 multipanel (12 months)

In [9]:
# Compute H′ anomalies relative to daily climatology
# Using groupby to align daily climatology (dayofyear) with the time dimension
Hprime500 = H500.groupby("time.dayofyear") - clim500

In [183]:
def composite_dates(ds, dates):
    """Mean composite over provided event dates."""
    dates = pd.to_datetime(dates)
    if len(dates) == 0:
        return None

    tmin = pd.to_datetime(ds.time.min().values)
    tmax = pd.to_datetime(ds.time.max().values)
    dates = dates[(dates >= tmin) & (dates <= tmax)]
    if len(dates) == 0:
        return None

    return ds.sel(time=ds.time.isin(dates)).mean("time", skipna=True)

def ttest_mask_within_month(Hprime, event_dates, month: int):
    """
    Welch t-test mask comparing event vs non-event days within the same calendar month.
    This avoids seasonality leakage when mapping significance.
    """
    event_dates = pd.to_datetime(event_dates)
    Hm = Hprime.sel(time=Hprime.time.dt.month == month)

    evt = Hm.sel(time=Hm.time.isin(event_dates))
    ctrl = Hm.sel(time=~Hm.time.isin(event_dates))

    if evt.time.size < 5 or ctrl.time.size < 5:
        return xr.zeros_like(Hm.isel(time=0), dtype=bool)

    t, p = ttest_ind(evt, ctrl, axis=0, equal_var=False, nan_policy="omit")
    return xr.DataArray(p < 0.05, coords=Hm.isel(time=0).coords)

def plot_monthly_multipanel(pol, events_by_month, Hprime, U, V, outname, label=None, threshold_note=None, period=None):
    """
    12-panel (3x4) monthly composite maps for a single pollutant.
    Shading: Z500' anomaly
    Contours: Z500' anomaly (solid positive, dashed negative)
    Vectors: mean winds at 500 hPa
    Stippling: p<0.05 (events vs non-events within month)
    """
    proj = ccrs.PlateCarree()
    fig, axes = plt.subplots(3, 4, figsize=(16, 9), dpi=250, subplot_kw={"projection": proj})
    axes = axes.flatten()

    # --- Precompute monthly anomalies to lock a consistent color scale
    Hp_month = {}
    max_abs = 0.0
    for m in range(1, 13):
        dates_m = events_by_month[m]
        Hp = composite_dates(Hprime, dates_m)
        Hp_month[m] = Hp
        if Hp is not None:
            max_abs = max(max_abs, float(np.nanmax(np.abs(Hp.values))))

    if max_abs == 0:
        raise ValueError("max_abs is zero. No events or missing data?")

    norm = TwoSlopeNorm(vcenter=0, vmin=-max_abs, vmax=max_abs)
    pcm_ref = None

    # --- Plot each month
    for i, m in enumerate(range(1, 13)):
        ax = axes[i]
        ax.set_extent([LON_MIN, LON_MAX, LAT_MIN, LAT_MAX], crs=proj)
        ax.coastlines(resolution="50m", linewidth=0.5)
        ax.add_feature(cfeature.BORDERS, linewidth=0.4)
        ax.add_feature(cfeature.STATES.with_scale("50m"), linewidth=0.3)

        dates_m = events_by_month[m]
        n_evt = len(dates_m)
        Hp = Hp_month[m]

        if Hp is None or n_evt == 0:
            ax.set_title(f"{calendar.month_abbr[m]} (n=0)", fontsize=10, weight="bold")
            ax.text(0.5, 0.5, "No events", transform=ax.transAxes,
                    ha="center", va="center", fontsize=10)
            continue

        Um = composite_dates(U, dates_m)
        Vm = composite_dates(V, dates_m)
        sig = ttest_mask_within_month(Hprime, dates_m, month=m)

        Hp_np = Hp.values
        Um_np = Um.values
        Vm_np = Vm.values
        sig_np = sig.values

        pcm = ax.pcolormesh(lon2d, lat2d, Hp_np, cmap="RdBu_r",
                            norm=norm, shading="auto", transform=proj)
        pcm_ref = pcm

        # contours (fixed step)
        stepc = 5
        lev = np.arange(-max_abs, max_abs + stepc, stepc)
        ax.contour(lon2d, lat2d, Hp_np, levels=lev[lev > 0],
                   colors="k", linewidths=0.4, linestyles="solid", transform=proj)
        ax.contour(lon2d, lat2d, Hp_np, levels=lev[lev < 0],
                   colors="k", linewidths=0.4, linestyles="dashed", transform=proj)

        # wind vectors
        step = 4
        ax.quiver(lon2d[::step, ::step], lat2d[::step, ::step],
                  Um_np[::step, ::step], Vm_np[::step, ::step],
                  scale=700, width=0.002, color="black", transform=proj)

        # stippling
        y, x = np.where(sig_np)
        thin = 8
        y = y[::thin]; x = x[::thin]
        ax.scatter(lon2d[y, x], lat2d[y, x], s=2, c="k", alpha=0.25, transform=proj)

        # MCMA box + CDMX star
        rect = mpatches.Rectangle((MCMA_BOX[0], MCMA_BOX[1]), MCMA_BOX[2], MCMA_BOX[3],
                                  fill=False, edgecolor="k", linewidth=1, transform=proj)
        ax.add_patch(rect)
        ax.plot(LON_CDMX, LAT_CDMX, marker="*", color="gold",
                markersize=8, markeredgecolor="k", transform=proj)

        # grid labels: only left column + bottom row
        gl = ax.gridlines(draw_labels=True, linewidth=0.2, color="gray", alpha=0.5, linestyle="--")
        gl.top_labels = False
        gl.right_labels = False
        if (i % 4) != 0:
            gl.left_labels = False
        if i < 8:
            gl.bottom_labels = False

        ax.set_title(f"{calendar.month_abbr[m]} (n={n_evt})", fontsize=10, weight="bold")

    # shared colorbar
    cbar_ax = fig.add_axes([0.92, 0.20, 0.015, 0.60])
    cb = fig.colorbar(pcm_ref, cax=cbar_ax)
    cb.set_label("Z500 anomaly (m)")

    # period (always "range of years")
    if period is None:
        all_dates = []
        for m in range(1, 13):
            dm = events_by_month.get(m, [])
            if dm is not None and len(dm) > 0:
                all_dates.extend(list(pd.to_datetime(dm)))
        if len(all_dates) > 0:
            y0 = pd.to_datetime(min(all_dates)).year
            y1 = pd.to_datetime(max(all_dates)).year
            period_str = f"{y0}–{y1}"
        else:
            period_str = "N/A"
    else:
        period_str = str(period)

    # build title parts
    parts = [f"{pol}: Monthly Z500′ composites", period_str]
    if label:
        parts.insert(1, f"({label})")
    if threshold_note:
        parts.append(f"— {threshold_note}")

    fig.suptitle(" ".join(parts), fontsize=14, weight="bold", y=0.98)

    plt.tight_layout(rect=[0, 0, 0.9, 0.95])
    plt.savefig(outname, dpi=300, bbox_inches="tight")
    plt.close(fig)
    print("Saved:", outname)


In [11]:
#p10
for pol in pollutants:
    plot_monthly_multipanel(
        pol=pol,
        events_by_month=events_p10[pol],
        Hprime=Hprime500,
        U=U500,
        V=V500,
        outname=f"Z500_monthly_p10_{pol.replace('.', '')}.png",
        label="p10"
    )

C:\Users\DELL\AppData\Local\Temp\ipykernel_1672\1809083599.py:137: UserWarning: This figure includes Axes that are not compatible with tight_layout, so results might be incorrect.
  plt.tight_layout(rect=[0, 0, 0.9, 0.95])


Saved: Z500_monthly_p10_PM25.png


C:\Users\DELL\AppData\Local\Temp\ipykernel_1672\1809083599.py:137: UserWarning: This figure includes Axes that are not compatible with tight_layout, so results might be incorrect.
  plt.tight_layout(rect=[0, 0, 0.9, 0.95])


Saved: Z500_monthly_p10_PM10.png


In [12]:
# p90
for pol in pollutants:
    plot_monthly_multipanel(
        pol=pol,
        events_by_month=events_p90[pol],
        Hprime=Hprime500,
        U=U500,
        V=V500,
        outname=f"Z500_monthly_p90_{pol.replace('.', '')}.png",
        label="p90"
    )

C:\Users\DELL\AppData\Local\Temp\ipykernel_1672\1809083599.py:137: UserWarning: This figure includes Axes that are not compatible with tight_layout, so results might be incorrect.
  plt.tight_layout(rect=[0, 0, 0.9, 0.95])


Saved: Z500_monthly_p90_PM25.png


C:\Users\DELL\AppData\Local\Temp\ipykernel_1672\1809083599.py:137: UserWarning: This figure includes Axes that are not compatible with tight_layout, so results might be incorrect.
  plt.tight_layout(rect=[0, 0, 0.9, 0.95])


Saved: Z500_monthly_p90_PM10.png


## 6A. Diurnal + semidiurnal harmonics (24h + 12h) from RAMA hourly data

In [20]:
DATA_DIR = Path("data-icaycc")
MISSING_VALUE = -99

# From setup section
YEARS_PM10 = range(HOURLY_PM10_START_YEAR, HOURLY_PM10_END_YEAR + 1)  # 1995–2023
YEARS_PM25 = range(HOURLY_PM25_START_YEAR, HOURLY_PM25_END_YEAR + 1)  # 2003–2023

POLLUTANT_TAGS = {
    "PM10": ["PM10"],
    "PM25": ["PM25", "PM2.5", "PM2_5"]
}

In [21]:
# -----------------------------
# Helpers
# -----------------------------
def zip_path_for_year(year: int) -> Path:
    yy = f"{year % 100:02d}"
    candidates = list(DATA_DIR.glob(f"{yy}RAMA*.zip"))
    if not candidates:
        candidates = list(DATA_DIR.rglob(f"{yy}RAMA*.zip"))
    if not candidates:
        raise FileNotFoundError(f"Zip for year={year} not found. Expected {yy}RAMA*.zip inside {DATA_DIR}")
    return sorted(candidates, key=lambda p: len(str(p)))[0]

def find_member_for_year(zip_obj: zipfile.ZipFile, year: int, pollutant: str) -> str:
    year_str = str(year)
    tags = POLLUTANT_TAGS[pollutant]
    members = zip_obj.namelist()

    patterns = []
    for tag in tags:
        patterns.append(re.compile(rf".*{year_str}.*{tag}.*\.(csv|xls|xlsx)$", re.IGNORECASE))

    matches = []
    for m in members:
        for pat in patterns:
            if pat.match(m):
                matches.append(m)

    if not matches:
        raise FileNotFoundError(
            f"No member found in zip for year={year}, pollutant={pollutant}. "
            f"Members example: {members[:10]}"
        )

    # Prefer xls/xlsx/csv order is not critical; keep shortest path and first match
    matches_sorted = sorted(matches, key=lambda s: (len(s), s.lower()))
    return matches_sorted[0]

def read_year_pollutant_from_zip(zip_path: Path, year: int, pollutant: str) -> pd.DataFrame:
    """
    Read one year file for a pollutant from the given zip.
    Returns a dataframe with columns:
      - dt (datetime)
      - hour (int 1–24 or 0–23 depending on source)
      - row_mean (float): hourly mean across stations (area-mean proxy)
    """
    with zipfile.ZipFile(zip_path, "r") as z:
        member = find_member_for_year(z, year, pollutant)

        # Prefer Excel read (my files are Excel 97-2003)
        with z.open(member) as f:
            if member.lower().endswith(".csv"):
                df = pd.read_csv(f, sep=None, engine="python")
            else:
                df = pd.read_excel(f)

    df.columns = [str(c).strip() for c in df.columns]

    if "FECHA" not in df.columns or "HORA" not in df.columns:
        raise ValueError(f"Expected FECHA and HORA in {zip_path.name}:{member}. Found: {df.columns[:10]}")

    df["FECHA"] = pd.to_datetime(df["FECHA"], dayfirst=True, errors="coerce")
    df["HORA"]  = pd.to_numeric(df["HORA"], errors="coerce")

    station_cols = [c for c in df.columns if c not in ["FECHA", "HORA"]]
    df[station_cols] = df[station_cols].replace(MISSING_VALUE, np.nan)

    # Hourly mean across stations (use this for harmonic climatology)
    df["row_mean"] = df[station_cols].mean(axis=1, skipna=True)

    out = df[["FECHA", "HORA", "row_mean"]].dropna(subset=["FECHA", "HORA"])
    out = out.rename(columns={"FECHA": "date", "HORA": "hour"})
    out["dt"] = out["date"] + pd.to_timedelta(out["hour"], unit="h")

    # Standardize hour-of-day to 1–24 for plotting
    # If hours are 0–23, convert to 1–24; if already 1–24, keep.
    if out["hour"].min() == 0 and out["hour"].max() == 23:
        out["hour_plot"] = out["hour"] + 1
    else:
        out["hour_plot"] = out["hour"]

    out["month"] = out["date"].dt.month
    out["year"]  = out["date"].dt.year

    return out[["dt", "date", "year", "month", "hour_plot", "row_mean"]]

In [22]:
# -----------------------------
# Build monthly-hourly climatology by year
# -----------------------------
def build_monthly_hourly_climatology(pollutant: str, years: range) -> pd.DataFrame:
    """
    Returns a tidy DataFrame:
      year, month, hour(1..24), mean_conc
    where mean_conc is hourly mean across stations averaged over all days of that month in that year.
    """
    rows = []

    for y in years:
        zpath = zip_path_for_year(y)
        dfy = read_year_pollutant_from_zip(zpath, y, pollutant)

        # Group to monthly-hourly mean (averaging over days)
        g = (
            dfy.dropna(subset=["row_mean"])
               .groupby(["year", "month", "hour_plot"])["row_mean"]
               .mean()
               .reset_index()
               .rename(columns={"hour_plot": "hour", "row_mean": "mean_conc"})
        )

        rows.append(g)

    out = pd.concat(rows, ignore_index=True)

    # Ensure full hour 1–24 exists per month-year (for smooth plotting)
    out["hour"] = out["hour"].astype(int)
    return out

In [23]:
# -----------------------------
# Harmonic fit: 24h + 12h
# -----------------------------
def fit_diurnal_harmonics(hours_1_24: np.ndarray, y: np.ndarray):
    """
    Fit y(h) = a0 + a1*cos(2πh/24) + b1*sin(2πh/24) + a2*cos(2πh/12) + b2*sin(2πh/12)
    Returns fitted yhat and coefficients.
    """
    h = hours_1_24.astype(float)

    w1 = 2 * np.pi / 24.0
    w2 = 2 * np.pi / 12.0

    X = np.column_stack([
        np.ones_like(h),
        np.cos(w1 * h), np.sin(w1 * h),
        np.cos(w2 * h), np.sin(w2 * h),
    ])

    # Solve least squares
    beta, *_ = np.linalg.lstsq(X, y, rcond=None)
    yhat = X @ beta
    return yhat, beta

In [130]:
# -----------------------------
# Plot: 12-panel monthly harmonic climatology
# -----------------------------
def plot_monthly_harmonic_climatology(df_clim, pollutant_label, year_start, year_end, out_png):
    """
    Create 12 panels (3x4):
      - thin colored lines: year-specific monthly hourly means
      - black thick line: harmonic fit (24h+12h) on multi-year mean
      - all panels share the same y-scale for easy comparison
    """
    fig, axes = plt.subplots(3, 4, figsize=(16, 9), dpi=250, sharex=True, sharey=False)
    axes = axes.flatten()

    hours = np.arange(1, 25)
    global_min, global_max = np.inf, -np.inf

    for i, m in enumerate(range(1, 13)):
        ax = axes[i]
        subm = df_clim[df_clim["month"] == m].copy()

        # Track global range across months (including all yearly curves)
        month_min = subm["mean_conc"].min() if not subm.empty else np.nan
        month_max = subm["mean_conc"].max() if not subm.empty else np.nan
        if np.isfinite(month_min):
            global_min = min(global_min, month_min)
        if np.isfinite(month_max):
            global_max = max(global_max, month_max)

        # Plot each year curve (thin)
        years = sorted(subm["year"].unique())
        highlight_years = years[-4:]  # last 4 available years
        for y in years:
            sy = subm[subm["year"] == y].set_index("hour").reindex(hours)
            if y in highlight_years:
                ax.plot(hours, sy["mean_conc"].values, linewidth=1, alpha=0.95, linestyle="--", label=str(y))  # keep visible
            else:
                ax.plot(hours, sy["mean_conc"].values, color="0.7", linewidth=0.8, alpha=0.25)  # faint gray

        # Multi-year mean + harmonic fit
        mean_all = subm.groupby("hour")["mean_conc"].mean().reindex(hours)
        y_mean = mean_all.values

        if np.isfinite(y_mean).sum() >= 10:
            yhat, _ = fit_diurnal_harmonics(hours, y_mean)
            ax.plot(hours, yhat, color="black", linewidth=1.4, label="Harmonic fit")
            month_max = np.nanmax([month_max, np.nanmax(yhat)])
            month_min = np.nanmin([month_min, np.nanmin(yhat)])
        else:
            ax.plot(hours, y_mean, color="black", linewidth=1.4, label="Mean (insufficient)")

        ax.set_title(f"Hourly mean {pollutant_label} in {calendar.month_name[m]}", fontsize=10, fontweight="bold")
        ax.set_xlim(1, 24)
        ax.set_xticks(np.arange(1, 25, 2))
        ax.grid(True, alpha=0.3)

        if i % 4 == 0:
            ax.set_ylabel(f"{pollutant_label} [µg/m³]")
        if i >= 8:
            ax.set_xlabel("Hour")

        # Update global min/max after harmonic
        if np.isfinite(month_min):
            global_min = min(global_min, month_min)
        if np.isfinite(month_max):
            global_max = max(global_max, month_max)

    # Apply shared y-limits across all subplots
    if np.isfinite(global_min) and np.isfinite(global_max):
        span = global_max - global_min
        pad = 0.05 * span if span > 0 else 1.0
        y_lo = max(0, global_min - pad)
        y_hi = global_max + pad
        for ax in axes:
            ax.set_ylim(y_lo, y_hi)

    # Global title and legend
    fig.suptitle(
        f"Hourly climatology by month with diurnal + semidiurnal harmonic fit ({year_start}–{year_end})",
        fontsize=14, fontweight="bold", y=0.98
    )

    # Build a clean legend: show only harmonic + a few recent years
    # Here: show harmonic and last 4 years if available
    handles, labels = axes[0].get_legend_handles_labels()
    # Keep harmonic fit always
    keep = [(h, l) for h, l in zip(handles, labels) if "Harmonic" in l or "fit" in l]
    # Add last years if present
    year_labels = sorted([l for l in labels if l.isdigit()])
    for yl in year_labels[-4:]:
        idx = labels.index(yl)
        keep.append((handles[idx], labels[idx]))

    # Add dummy line for "other years" (gray)
    dummy_line, = axes[0].plot([], [], color="0.7", linewidth=0.8, alpha=0.25, label="Other years")
    keep.append((dummy_line, "Rest of the years"))

    if keep:
        fig.legend([k[0] for k in keep], [k[1] for k in keep],
                   loc="upper left", bbox_to_anchor=(0.01, 0.98), frameon=False,
                   fontsize=9, ncol=2)

    plt.tight_layout(rect=[0, 0, 1, 0.95])
    plt.savefig(out_png, dpi=300, bbox_inches="tight")
    plt.close(fig)
    print(f"Saved: {out_png}")

In [134]:
# -----------------------------
# Run: PM10 and PM2.5
# -----------------------------
df_hclim_pm10 = build_monthly_hourly_climatology("PM10", YEARS_PM10)
plot_monthly_harmonic_climatology(
    df_hclim_pm10,
    pollutant_label="PM$_{10}$",
    year_start=HOURLY_PM10_START_YEAR,
    year_end=HOURLY_PM10_END_YEAR,
    out_png="PM10_hourly_climatology_harmonics_12panel.png"
)

df_hclim_pm25 = build_monthly_hourly_climatology("PM25", YEARS_PM25)
plot_monthly_harmonic_climatology(
    df_hclim_pm25,
    pollutant_label="PM$_{2.5}$",
    year_start=HOURLY_PM25_START_YEAR,
    year_end=HOURLY_PM25_END_YEAR,
    out_png="PM25_hourly_climatology_harmonics_12panel.png"
)

Saved: PM10_hourly_climatology_harmonics_12panel.png
Saved: PM25_hourly_climatology_harmonics_12panel.png


In [126]:
# -----------------------------
 # Print diurnal + semidiurnal harmonic coefficients (24h + 12h)
# -----------------------------
def summarize_diurnal_harmonics(df_clim, label):
    from IPython.display import display, HTML
    hours = np.arange(1, 25)
    rows = []
    for m in range(1, 13):
        subm = df_clim[df_clim["month"] == m]
        y_mean = subm.groupby("hour")["mean_conc"].mean().reindex(hours)
        y = y_mean.values
        if np.isfinite(y).sum() < 10:
            continue
        _, beta = fit_diurnal_harmonics(hours, y)
        a0, a1, b1, a2, b2 = beta
        rows.append({
            "month": calendar.month_abbr[m],
            "a0": a0,
            "a1": a1,
            "b1": b1,
            "a2": a2,
            "b2": b2,
            "amp24h": np.sqrt(a1**2 + b1**2),
            "amp12h": np.sqrt(a2**2 + b2**2)
        })
    if not rows:
        print(f"No valid monthly means for {label}")
        return
    df_out = pd.DataFrame(rows).round(3)
    display(HTML(f"<h3 style='margin:4px 0;'>{label}</h3>"))
    display(df_out)
    return df_out

In [129]:
pm10_harmonics = summarize_diurnal_harmonics(df_hclim_pm10, "Diurnal + semidiurnal coefficients for PM10 diurnal harmonics")
pm25_harmonics = summarize_diurnal_harmonics(df_hclim_pm25, "Diurnal + semidiurnal coefficients for PM2.5 diurnal harmonics")

,month,a0,a1,b1,a2,b2,amp24h,amp12h
0,Jan,63.70,-9.42,-5.03,1.86,-12.93,10.68,13.06
1,Feb,62.99,-9.43,-7.81,-0.05,-14.09,12.25,14.09
2,Mar,62.11,-8.44,-9.52,-4.21,-12.05,12.72,12.76
3,Apr,61.01,-9.24,-4.88,-6.65,-11.22,10.45,13.05
4,May,57.98,-8.74,-4.60,-5.82,-12.31,9.87,13.62
5,Jun,40.85,-8.77,-5.58,-3.35,-7.25,10.40,7.98
6,Jul,35.97,-10.88,-4.16,-1.16,-5.58,11.65,5.70
7,Aug,32.96,-10.35,-3.96,-1.16,-5.67,11.08,5.79
8,Sep,32.07,-9.59,-4.08,-1.27,-5.22,10.43,5.37
9,Oct,39.35,-11.12,-4.01,-2.14,-8.13,11.82,8.41


,month,a0,a1,b1,a2,b2,amp24h,amp12h
0,Jan,28.28,-5.39,3.18,2.33,-1.69,6.26,2.88
1,Feb,25.39,-4.89,1.76,1.97,-2.13,5.20,2.90
2,Mar,25.12,-4.45,1.94,1.22,-2.26,4.85,2.57
3,Apr,28.21,-4.53,2.75,0.51,-2.61,5.30,2.66
4,May,29.18,-5.54,2.50,1.22,-2.25,6.08,2.55
5,Jun,19.89,-4.61,1.30,1.78,-1.36,4.79,2.24
6,Jul,18.02,-5.93,0.26,2.49,-0.82,5.94,2.62
7,Aug,17.04,-5.42,0.06,2.10,-0.77,5.42,2.23
8,Sep,16.96,-4.31,-0.36,1.72,-0.60,4.33,1.82
9,Oct,19.19,-5.36,0.90,1.46,-1.49,5.43,2.08


## 6B. Annual + semiannual harmonics (365d + 182.5d) on daily series

In [ ]:
def fit_annual_harmonics(doy: np.ndarray, y: np.ndarray):
    """
    Fit y(d) = a0 + a1*cos(2πd/365) + b1*sin(2πd/365) + a2*cos(4πd/365) + b2*sin(4πd/365)
    (annual + semiannual).
    """
    d = doy.astype(float)
    w1 = 2 * np.pi / 365.0
    w2 = 4 * np.pi / 365.0  # semiannual

    X = np.column_stack([
        np.ones_like(d),
        np.cos(w1 * d), np.sin(w1 * d),
        np.cos(w2 * d), np.sin(w2 * d),
    ])

    beta, *_ = np.linalg.lstsq(X, y, rcond=None)
    yhat = X @ beta
    return yhat, beta

def decompose_harmonics(doy: np.ndarray, beta: np.ndarray):
    """
    Reconstructs the annual (1 cycle/year) and semi-annual (2 cycles/year) 
    components from the harmonic adjustment coefficients.

    beta = [a0, a1, b1, a2, b2]
    """
    w1 = 2 * np.pi / 365.0   # annual
    w2 = 4 * np.pi / 365.0   # semiannual

    a0, a1, b1, a2, b2 = beta

    # Annual component includes the mean level a0
    y_annual = a0 + a1 * np.cos(w1 * doy) + b1 * np.sin(w1 * doy)
    # Semiannual component (without a0)
    y_semi   = a2 * np.cos(w2 * doy) + b2 * np.sin(w2 * doy)

    return y_annual, y_semi

def plot_seasonal_harmonics_daily(df_daily, pol, out_png):
    """
    df_daily: indexed by datetime, daily mean series.
    """
    s = df_daily[pol].dropna().copy()
    s = s[~((s.index.month == 2) & (s.index.day == 29))] # remove leap day

    # Multi-year daily climatology by day-of-year
    clim = s.groupby(s.index.dayofyear).mean()
    doy = clim.index.values
    y = clim.values

    # Adjust + decompose
    yhat, beta = fit_annual_harmonics(doy, y)
    y_annual, y_semi = decompose_harmonics(doy, beta)

    # ========== Printing coefficients ==========
    a0, a1, b1, a2, b2 = beta
    print(f"\n{'='*60}")
    print(f"Harmonic coefficients for {pol}")
    print(f"{'='*60}")
    print(f"Mean level (a₀): {a0:>8.3f} µg/m³")
    print(f"Annual amplitude: {np.sqrt(a1**2 + b1**2):>8.3f} µg/m³")
    print(f"a₁ (annual cosine): {a1:>8.3f}")
    print(f"b₁ (annual sine): {b1:>8.3f}")
    print(f"Semiannual amplitude: {np.sqrt(a2**2 + b2**2):>8.3f} µg/m³")
    print(f"a₂ (semiannual cosine): {a2:>8.3f}")
    print(f"b₂ (semiannual sine): {b2:>8.3f}")
    print(f"{'='*60}\n")
    # ============================================
    
    fig, ax = plt.subplots(figsize=(10, 5), dpi=220)
    ax.plot(doy, y, color="tab:blue", linewidth=0.8, ls="--", label="Daily climatology", alpha=0.8)
    ax.plot(doy, y_annual, color="tab:green", linewidth=1.2, label="Annual (1 cycle)")
    ax.plot(doy, y_semi,   color="tab:red",   linewidth=1.2, label="Semiannual (2 cycles)")
    ax.plot(doy, y_annual + y_semi, color="k", linewidth=2, label="Annual+semiannual fit", alpha=0.85)

    # Bottom axis: Day of year (ensure no 0 tick and align with months)
    ax.set_xlim(1, 365) # non-leap
    ax.set_xlabel("Day of year", fontsize=11)
    ax.set_ylabel("Concentration [µg/m³]", fontsize=11)
    ax.grid(True, alpha=0.3, ls="-", lw=0.5)
    ax.legend(frameon=False, fancybox=True, shadow=True, loc="upper right", fontsize=10)

    # Top axis: Months (aligned to the same x-limits)
    ax2 = ax.twiny()
    ax2.set_xlim(ax.get_xlim()) # critical for alignment
    month_starts = pd.date_range("2001-01-01", "2002-01-01", freq="MS")[:-1] # non-leap year
    month_doys = month_starts.dayofyear.values
    month_labels = [d.strftime("%b") for d in month_starts]
    ax2.set_xticks(month_doys)
    ax2.set_xticklabels(month_labels)
    ax2.xaxis.set_ticks_position("top")
    ax2.xaxis.set_label_position("top")
    ax2.set_xlabel("Month")
    for md in month_doys: # lighter month grid lines and smaller top labels
        ax.axvline(md, color="gray", lw=0.3, alpha=0.15)
    ax2.tick_params(axis="x", labelsize=9)

    # Figure-level title (avoids overlap with top month axis)
    fig.suptitle(
        f"{pol}: seasonal cycle decomposition (annual + semiannual harmonics)",
        fontsize=12, fontweight="bold", y=0.99 # raise slightly to add breathing room
    )

    plt.tight_layout(rect=[0, 0, 1, 0.96]) # leaves ~4% at top for title
    plt.tight_layout()
    plt.savefig(out_png, dpi=300, bbox_inches="tight")
    plt.close(fig)
    print(f"Saved: {out_png}")

In [101]:
# Run for city-mean daily dataset (2012–2024)
plot_seasonal_harmonics_daily(df, "PM2.5", "PM25_seasonal_harmonics_annual_semiannual.png")
plot_seasonal_harmonics_daily(df, "PM10",  "PM10_seasonal_harmonics_annual_semiannual.png")


Harmonic coefficients for PM2.5
  Mean level (a₀):                23.036 µg/m³
  Annual amplitude:                 5.125 µg/m³
    a₁ (annual cosine):             3.164
    b₁ (annual sine):               4.033
  Semiannual amplitude:             2.669 µg/m³
    a₂ (semiannual cosine):         1.215
    b₂ (semiannual sine):          -2.376

Saved: PM25_seasonal_harmonics_annual_semiannual.png

Harmonic coefficients for PM10
  Mean level (a₀):                44.921 µg/m³
  Annual amplitude:                13.794 µg/m³
    a₁ (annual cosine):             9.940
    b₁ (annual sine):               9.563
  Semiannual amplitude:             2.936 µg/m³
    a₂ (semiannual cosine):         1.752
    b₂ (semiannual sine):          -2.356

Saved: PM10_seasonal_harmonics_annual_semiannual.png


## 7. NOM-172 (12h NowCast): weighted 12h moving average + categories (From Jan 2026)

In [106]:
# -----------------------------
# Paths and constants
# -----------------------------
DATA_DIR = Path("data-icaycc")   # folder containing RAMA zips
OUT_DIR  = Path(".")
OUT_DIR.mkdir(exist_ok=True)

MISSING_VALUE = -99

# Periods available in RAMA files you described
YEARS_PM10 = range(1995, 2024)  # 1995–2023
YEARS_PM25 = range(2003, 2024)  # 2003–2023

# NOM-172 adjustment factors (Annex A)
FA = {"PM10": 0.714, "PM25": 0.694}

# 2026 thresholds (IAYS / NOM-172 tables, "From Jan 2026")
# We will define an "event" as >= Poor (i.e., above "Acceptable")
THR_2026 = {
    "PM10": {"Good": 45, "Acceptable": 50, "Poor": 132, "VeryPoor": 213},  # Extreme >213
    "PM25": {"Good": 15, "Acceptable": 25, "Poor": 79,  "VeryPoor": 130},  # Extreme >130
}

In [107]:
# -----------------------------
# Helpers: locate zip + member file
# -----------------------------
def zip_path_for_year(year: int) -> Path:
    yy = f"{year % 100:02d}"
    candidates = list(DATA_DIR.glob(f"{yy}RAMA*.zip"))
    if not candidates:
        candidates = list(DATA_DIR.rglob(f"{yy}RAMA*.zip"))
    if not candidates:
        raise FileNotFoundError(f"Zip for year={year} not found (expected {yy}RAMA.zip) in {DATA_DIR}")
    return sorted(candidates, key=lambda p: len(str(p)))[0]

def find_member(zip_obj: zipfile.ZipFile, year: int, pollutant: str) -> str:
    year_str = str(year)
    tag = pollutant  # "PM10" or "PM25"
    members = zip_obj.namelist()

    # Prefer Excel 97-2003 and also allow CSV
    pat = re.compile(rf".*{year_str}.*{tag}.*\.(xls|xlsx|csv)$", re.IGNORECASE)
    matches = [m for m in members if pat.match(m)]
    if not matches:
        raise FileNotFoundError(f"No member found for {year} {pollutant} in zip. Example members: {members[:10]}")
    # Prefer xls if present
    matches = sorted(matches, key=lambda s: (0 if s.lower().endswith(".xls") else 1, len(s)))
    return matches[0]

def read_year_hourly_area_max(year: int, pollutant: str) -> pd.DataFrame:
    """
    Reads one year RAMA file and returns an hourly time series:
      - timestamp (datetime)
      - area_max (max across stations at that hour)
    Assumes columns: FECHA, HORA, station1, station2, ...
    Handles HORA in 1..24 (24 -> 00:00 next day).
    """
    zpath = zip_path_for_year(year)
    with zipfile.ZipFile(zpath, "r") as z:
        member = find_member(z, year, pollutant)
        with z.open(member) as f:
            if member.lower().endswith(".csv"):
                df = pd.read_csv(f, sep=None, engine="python")
            else:
                df = pd.read_excel(f)

    df.columns = [str(c).strip() for c in df.columns]
    if "FECHA" not in df.columns or "HORA" not in df.columns:
        raise ValueError(f"Expected FECHA and HORA in {zpath.name}:{member}")

    df["FECHA"] = pd.to_datetime(df["FECHA"], dayfirst=True, errors="coerce")
    df["HORA"]  = pd.to_numeric(df["HORA"], errors="coerce")

    station_cols = [c for c in df.columns if c not in ["FECHA", "HORA"]]
    df[station_cols] = df[station_cols].replace(MISSING_VALUE, np.nan)

    # Max across stations (hourly "worst station" / area maximum)
    df["area_max"] = df[station_cols].max(axis=1, skipna=True)

    out = df[["FECHA", "HORA", "area_max"]].dropna(subset=["FECHA", "HORA"])
    out = out.rename(columns={"FECHA": "date", "HORA": "hour"})

    # Build timestamps (HORA: 1..24)
    # hour==24 is treated as 00:00 of next day
    out["hour_int"] = out["hour"].astype(int)
    out["date_adj"] = out["date"] + pd.to_timedelta((out["hour_int"] == 24).astype(int), unit="D")
    out["hour_adj"] = out["hour_int"].where(out["hour_int"] < 24, 0)

    out["timestamp"] = out["date_adj"] + pd.to_timedelta(out["hour_adj"], unit="H")
    out = out[["timestamp", "area_max"]].sort_values("timestamp").reset_index(drop=True)
    return out

def build_hourly_series(pollutant: str, years: range) -> pd.DataFrame:
    parts = []
    for y in years:
        try:
            parts.append(read_year_hourly_area_max(y, pollutant))
        except Exception as e:
            print(f"[WARN] {pollutant} {y}: {e}")
    if not parts:
        raise RuntimeError(f"No hourly data built for {pollutant}. Check files and paths.")
    out = pd.concat(parts, ignore_index=True).sort_values("timestamp")
    # If duplicates exist, keep the max
    out = out.groupby("timestamp", as_index=False)["area_max"].max()
    return out

In [108]:
# -----------------------------
# NOM NowCast 12h (Annex A)
# -----------------------------
def nowcast_12h(series: pd.Series, pollutant: str) -> pd.Series:
    """
    Computes hourly NowCast 12h using Annex A rules:
      - Uses last 12 hours including current
      - Valid only if at least 2 of last 3 hours are present
      - W = 1 - (Cmax - Cmin)/Cmax = Cmin/Cmax, forced to [0.5, 1.0], rounded to 2 decimals
      - Weighted average with W^(i-1) where i=1 is most recent
      - Multiplies by FA[pollutant]
      - Returns float series (not rounded to int)
    """
    fa = FA[pollutant]
    x = series.astype(float)

    # Pre-allocate output
    out = pd.Series(index=x.index, dtype=float)

    # We assume index is regular hourly; if missing timestamps, it is ok (rolling uses existing)
    # We will explicitly pull a 12-hour window by timestamp using reindex.
    idx = x.index

    for t in idx:
        # Build exact 12-hour window ending at t (t, t-1h, ..., t-11h)
        window_idx = pd.date_range(end=t, periods=12, freq="H")
        wv = x.reindex(window_idx)

        # Validity: at least 2 of last 3 hours present (most recent = last element in window_idx)
        last3 = wv.iloc[-3:]
        if last3.notna().sum() < 2:
            out.loc[t] = np.nan
            continue

        # Reverse so that i=1 is most recent (Annex A convention)
        c = wv.iloc[::-1].values  # length 12, c[0]=most recent

        # Compute Cmax/Cmin ignoring NaN
        cmax = np.nanmax(c)
        cmin = np.nanmin(c)

        if not np.isfinite(cmax) or cmax <= 0:
            out.loc[t] = np.nan
            continue

        w_raw = 1.0 - (cmax - cmin) / cmax  # = cmin/cmax
        # Force to [0.5, 1.0] and round to 2 decimals
        W = max(0.5, min(1.0, round(float(w_raw), 2)))

        # Weighted average, skipping missing hours but keeping exponents positionally
        num = 0.0
        den = 0.0
        for i in range(12):  # i=0 -> exponent 0 (most recent)
            if np.isfinite(c[i]):
                weight = (W ** i)
                num += c[i] * weight
                den += weight

        if den == 0:
            out.loc[t] = np.nan
            continue

        out.loc[t] = fa * (num / den)

    return out

In [112]:
# -----------------------------
# Classification (2026 thresholds)
# -----------------------------
def classify_2026(value: float, pollutant: str) -> str:
    if not np.isfinite(value):
        return "Missing"
    thr = THR_2026[pollutant]
    if value <= thr["Good"]:
        return "Good"
    elif value <= thr["Acceptable"]:
        return "Acceptable"
    elif value <= thr["Poor"]:
        return "Poor"
    elif value <= thr["VeryPoor"]:
        return "Very Poor"
    else:
        return "Extremely Poor"

def daily_max_and_hour(df_hourly: pd.DataFrame, col: str) -> pd.DataFrame:
    """
    From an hourly series, returns per-day:
      - daily_max
      - hour_of_max
    """
    tmp = df_hourly.copy()
    tmp["date"] = tmp["timestamp"].dt.floor("D")
    
    # Guard: if everything is NaN, return empty to avoid idxmax producing NaN indexes.
    if tmp[col].notna().sum() == 0:
        return pd.DataFrame(columns=["date", "timestamp", "daily_max", "hour_of_max"])
    
    idx = tmp.groupby("date")[col].idxmax()
    idx = idx[idx.notna()].astype(int)  # drop days with all-NaN
    if len(idx) == 0:
        return pd.DataFrame(columns=["date", "timestamp", "daily_max", "hour_of_max"])
    
    out = tmp.loc[idx, ["date", "timestamp", col]].copy()
    out = out.rename(columns={col: "daily_max"})
    out["hour_of_max"] = out["timestamp"].dt.hour
    out = out.sort_values("date").reset_index(drop=True)
    return out

In [113]:
# -----------------------------
# Run pipeline for PM10 and PM2.5
# -----------------------------
def run_nom_nowcast_pipeline(pollutant: str, years: range, out_prefix: str):
    # 1) Build hourly area_max series
    dfh = build_hourly_series(pollutant, years)
    dfh = dfh.sort_values("timestamp").reset_index(drop=True)
    dfh = dfh.set_index("timestamp")

    # 2) Compute NowCast 12h
    dfh["nowcast12h"] = nowcast_12h(dfh["area_max"], pollutant)

    # 3) Classify (2026)
    dfh["category_2026"] = dfh["nowcast12h"].apply(lambda v: classify_2026(v, pollutant))

    # 4) Export hourly series
    hourly_out = dfh.reset_index()
    hourly_out.to_csv(f"{out_prefix}_hourly_nowcast12h.csv", index=False)
    print(f"Saved: {out_prefix}_hourly_nowcast12h.csv")

    # 5) Daily maxima of nowcast + hour of occurrence
    dfd = daily_max_and_hour(hourly_out, "nowcast12h")
    dfd["category_2026"] = dfd["daily_max"].apply(lambda v: classify_2026(v, pollutant))

    # Define "event day" as >= Poor (i.e., above Acceptable threshold)
    # PM10: event if daily_max > 50
    # PM25: event if daily_max > 25
    thr_event = THR_2026[pollutant]["Acceptable"]
    dfd["is_event_ge_poor"] = dfd["daily_max"] > thr_event

    dfd.to_csv(f"{out_prefix}_daily_max_nowcast12h.csv", index=False)
    print(f"Saved: {out_prefix}_daily_max_nowcast12h.csv")

    return hourly_out, dfd

In [114]:
# PM10
pm10_hourly, pm10_daily = run_nom_nowcast_pipeline(
    pollutant="PM10",
    years=YEARS_PM10,
    out_prefix="PM10_NOM172_2026"
)

# PM2.5 (files are PM25)
pm25_hourly, pm25_daily = run_nom_nowcast_pipeline(
    pollutant="PM25",
    years=YEARS_PM25,
    out_prefix="PM25_NOM172_2026"
)

C:\Users\DELL\AppData\Local\Temp\ipykernel_1672\4039281186.py:66: FutureWarning: 'H' is deprecated and will be removed in a future version. Please use 'h' instead of 'H'.
  out["timestamp"] = out["date_adj"] + pd.to_timedelta(out["hour_adj"], unit="H")
C:\Users\DELL\AppData\Local\Temp\ipykernel_1672\4039281186.py:66: FutureWarning: 'H' is deprecated and will be removed in a future version. Please use 'h' instead of 'H'.
  out["timestamp"] = out["date_adj"] + pd.to_timedelta(out["hour_adj"], unit="H")
C:\Users\DELL\AppData\Local\Temp\ipykernel_1672\4039281186.py:66: FutureWarning: 'H' is deprecated and will be removed in a future version. Please use 'h' instead of 'H'.
  out["timestamp"] = out["date_adj"] + pd.to_timedelta(out["hour_adj"], unit="H")
C:\Users\DELL\AppData\Local\Temp\ipykernel_1672\4039281186.py:66: FutureWarning: 'H' is deprecated and will be removed in a future version. Please use 'h' instead of 'H'.
  out["timestamp"] = out["date_adj"] + pd.to_timedelta(out["hour_adj"]

Saved: PM10_NOM172_2026_hourly_nowcast12h.csv
Saved: PM10_NOM172_2026_daily_max_nowcast12h.csv


C:\Users\DELL\AppData\Local\Temp\ipykernel_1672\4039281186.py:66: FutureWarning: 'H' is deprecated and will be removed in a future version. Please use 'h' instead of 'H'.
  out["timestamp"] = out["date_adj"] + pd.to_timedelta(out["hour_adj"], unit="H")
C:\Users\DELL\AppData\Local\Temp\ipykernel_1672\4039281186.py:66: FutureWarning: 'H' is deprecated and will be removed in a future version. Please use 'h' instead of 'H'.
  out["timestamp"] = out["date_adj"] + pd.to_timedelta(out["hour_adj"], unit="H")
C:\Users\DELL\AppData\Local\Temp\ipykernel_1672\4039281186.py:66: FutureWarning: 'H' is deprecated and will be removed in a future version. Please use 'h' instead of 'H'.
  out["timestamp"] = out["date_adj"] + pd.to_timedelta(out["hour_adj"], unit="H")
C:\Users\DELL\AppData\Local\Temp\ipykernel_1672\4039281186.py:66: FutureWarning: 'H' is deprecated and will be removed in a future version. Please use 'h' instead of 'H'.
  out["timestamp"] = out["date_adj"] + pd.to_timedelta(out["hour_adj"]

Saved: PM25_NOM172_2026_hourly_nowcast12h.csv
Saved: PM25_NOM172_2026_daily_max_nowcast12h.csv


In [118]:
# ============================================================
# Build NOM-based event dates (2012–2023) for composites
# ============================================================

NOM_START = "2012-01-01"
NOM_END   = "2023-12-31"   # RAMA ends 2023

def build_events_by_month_from_nom(daily_df, start=NOM_START, end=NOM_END,
                                   event_mode="ge_extreme"):
    """
    Returns dict month(int 1-12) -> DatetimeIndex of NOM event days.

    event_mode:
      - "ge_poor": uses the precomputed boolean is_event_ge_poor (>= Poor, i.e. above Acceptable)
      - "ge_very_poor": Very Poor + Extremely Poor
      - "ge_extreme": Extremely Poor only
    """
    d = daily_df.copy()
    d["date"] = pd.to_datetime(d["date"])
    d = d[(d["date"] >= pd.to_datetime(start)) & (d["date"] <= pd.to_datetime(end))]

    if event_mode == "ge_poor":
        ev = d[d["is_event_ge_poor"]]
    elif event_mode == "ge_very_poor":
        ev = d[d["category_2026"].isin(["Very Poor", "Extremely Poor"])]
    elif event_mode == "ge_extreme":
        ev = d[d["category_2026"].isin(["Extremely Poor"])]
    else:
        raise ValueError("event_mode must be one of: ge_poor, ge_very_poor, ge_extreme")

    out = {m: [] for m in range(1, 13)}
    for m, g in ev.groupby(ev["date"].dt.month):
        out[int(m)] = pd.DatetimeIndex(sorted(g["date"].dt.normalize().unique()))

    return out, ev

# Build NOM events
EVENT_MODE = "ge_extreme"  # could be also "ge_very_poor" or "ge_poor"

events_nom_pm10, ev_pm10 = build_events_by_month_from_nom(pm10_daily, event_mode=EVENT_MODE)
events_nom_pm25, ev_pm25 = build_events_by_month_from_nom(pm25_daily, event_mode=EVENT_MODE)

# Quick counts
print("NOM event counts (2012–2023):")
print("PM10 total:", len(ev_pm10), "| by month:", {m: len(events_nom_pm10[m]) for m in range(1,13)})
print("PM2.5 total:", len(ev_pm25), "| by month:", {m: len(events_nom_pm25[m]) for m in range(1,13)})

NOM event counts (2012–2023):
PM10 total: 70 | by month: {1: 19, 2: 3, 3: 12, 4: 10, 5: 9, 6: 0, 7: 1, 8: 0, 9: 1, 10: 0, 11: 0, 12: 15}
PM2.5 total: 18 | by month: {1: 7, 2: 2, 3: 0, 4: 2, 5: 0, 6: 0, 7: 0, 8: 0, 9: 0, 10: 0, 11: 0, 12: 7}


In [119]:
# ============================================================
# Running the same composite plotter but using NOM events
# ============================================================

# PM10 NOM composites
plot_monthly_multipanel(
    pol="PM10 (NOM NowCast12h events)",
    events_by_month=events_nom_pm10,
    Hprime=Hprime500,
    U=U500,
    V=V500,
    outname=f"Z500_monthly_NOM_{EVENT_MODE}_PM10.png",
    label=f"NOM {EVENT_MODE}"
)

# PM2.5 NOM composites
plot_monthly_multipanel(
    pol="PM2.5 (NOM NowCast12h events)",
    events_by_month=events_nom_pm25,
    Hprime=Hprime500,
    U=U500,
    V=V500,
    outname=f"Z500_monthly_NOM_{EVENT_MODE}_PM25.png",
    label=f"NOM {EVENT_MODE}"
)

C:\Users\DELL\AppData\Local\Temp\ipykernel_1672\1809083599.py:137: UserWarning: This figure includes Axes that are not compatible with tight_layout, so results might be incorrect.
  plt.tight_layout(rect=[0, 0, 0.9, 0.95])


Saved: Z500_monthly_NOM_ge_extreme_PM10.png


C:\Users\DELL\AppData\Local\Temp\ipykernel_1672\1809083599.py:137: UserWarning: This figure includes Axes that are not compatible with tight_layout, so results might be incorrect.
  plt.tight_layout(rect=[0, 0, 0.9, 0.95])


Saved: Z500_monthly_NOM_ge_extreme_PM25.png


---

In [ ]:
#-----------------------------------------------------------------------------------------------------------------------------
# Author:             Erick Rico Esparza
# Dates:              Jan 27- Feb 2, 2026
# Description:        A methodological sensitivity check to finalize event definition and temporal resolution
# Organization:       Tampere University / Institute of Atmospheric Sciences and Climate Change (ICAyCC-UNAM)
#-----------------------------------------------------------------------------------------------------------------------------

# Stage 1.3

In [ ]:
# PCAA (contingency) thresholds in µg/m³
PCAA_THR = {
    "PM10": {"phase1": 214.0, "phase2": 354.0},
    "PM25": {"phase1": 97.4,  "phase2": 150.4},
}

PCAA_START = "2012-01-01"
PCAA_END   = "2023-12-31"  # RAMA ends 2023

## 1. Diurnal + semidiurnal harmonics (ALL years, last 4 highlighted) + thresholds

In [164]:
def plot_monthly_harmonic_climatology_new(df_clim, pollutant_key, pollutant_label, year_start, year_end, out_png):
    """
    12 panels:
      - ALL years shown (same style)
      - harmonic fit in black computed from multi-year mean (24h+12h)
      - thresholds: Phase I (red, solid) + Phase II (blue, solid)
      - shared y-limits across 12 panels, forced to include thresholds
    """

    fig, axes = plt.subplots(3, 4, figsize=(16, 9), dpi=250, sharex=True, sharey=False)
    axes = axes.flatten()

    hours = np.arange(1, 25)
    color_cycle = plt.rcParams['axes.prop_cycle'].by_key().get('color', None)

    thr1 = PCAA_THR[pollutant_key]["phase1"]
    thr2 = PCAA_THR[pollutant_key]["phase2"]

    # First pass: compute global min/max INCLUDING thresholds so they are always visible
    global_min, global_max = np.inf, -np.inf

    for m in range(1, 13):
        subm = df_clim[df_clim["month"] == m]

        if not subm.empty:
            global_min = min(global_min, np.nanmin(subm["mean_conc"].values))
            global_max = max(global_max, np.nanmax(subm["mean_conc"].values))

            mean_all = subm.groupby("hour")["mean_conc"].mean().reindex(hours)
            y_mean = mean_all.values
            if np.isfinite(y_mean).sum() >= 10:
                yhat, _ = fit_diurnal_harmonics(hours, y_mean)
                global_min = min(global_min, np.nanmin(yhat))
                global_max = max(global_max, np.nanmax(yhat))

    # Force include thresholds (zoom out)
    global_min = min(global_min, thr1, thr2)
    global_max = max(global_max, thr1, thr2)

    # Add padding
    if np.isfinite(global_min) and np.isfinite(global_max):
        span = global_max - global_min
        pad = 0.08 * span if span > 0 else 5.0
        y_lo = max(0, global_min - pad)
        y_hi = global_max + pad
    else:
        y_lo, y_hi = 0, max(thr1, thr2) * 1.2

    # Second pass: plot
    for i, m in enumerate(range(1, 13)):
        ax = axes[i]
        subm = df_clim[df_clim["month"] == m].copy()

        years = sorted(subm["year"].unique())
        for idx, y in enumerate(years):
            sy = subm[subm["year"] == y].set_index("hour").reindex(hours)
            ax.plot(
                hours,
                sy["mean_conc"].values,
                linewidth=1.0,
                alpha=0.85,
                linestyle="--",
                color=(color_cycle[idx % len(color_cycle)] if color_cycle else None),
            )

        # Harmonic fit on multi-year mean
        mean_all = subm.groupby("hour")["mean_conc"].mean().reindex(hours)
        y_mean = mean_all.values
        if np.isfinite(y_mean).sum() >= 10:
            yhat, _ = fit_diurnal_harmonics(hours, y_mean)
            ax.plot(hours, yhat, color="black", linewidth=1.8, label="Harmonic fit")
        else:
            ax.plot(hours, y_mean, color="black", linewidth=1.8, label="Mean (insufficient)")

        # Thresholds (dashed, visible colors)
        ax.axhline(thr1, color="red",  linewidth=2, alpha=0.95, linestyle="--")
        ax.axhline(thr2, color="blue", linewidth=2, alpha=0.95, linestyle="--")

        ax.set_ylim(y_lo, y_hi)
        ax.set_title(f"Hourly mean {pollutant_label} in {calendar.month_name[m]}", fontsize=10, fontweight="bold")
        ax.set_xlim(1, 24)
        ax.set_xticks(np.arange(1, 25, 2))
        ax.grid(True, alpha=0.3)

        if i % 4 == 0:
            ax.set_ylabel(f"{pollutant_label} [µg/m³]")
        if i >= 8:
            ax.set_xlabel("Hour")

    fig.suptitle(
        f"Hourly climatology by month with diurnal + semidiurnal harmonic fit ({year_start}–{year_end})\n"
        f"Thresholds: Phase I and Phase II",
        fontsize=13, fontweight="bold", y=0.98
    )

    legend_items = [
        Line2D([0],[0], color="black", lw=1.8, label="Harmonic fit (24h+12h)"),
        Line2D([0],[0], color="red",  lw=2, ls="--", label=f"Phase I = {thr1:g} µg/m³"),
        Line2D([0],[0], color="blue", lw=2, ls="--", label=f"Phase II = {thr2:g} µg/m³"),
        Line2D([0],[0], color="0.3",  lw=1.0, ls="--", label="All years"),
    ]
    fig.legend(handles=legend_items, loc="upper left", bbox_to_anchor=(0.01, 0.98), frameon=False, fontsize=9)

    plt.tight_layout(rect=[0, 0, 1, 0.93])
    plt.savefig(out_png, dpi=300, bbox_inches="tight")
    plt.close(fig)
    print(f"Saved: {out_png}")

In [165]:
plot_monthly_harmonic_climatology_new(
    df_hclim_pm10, pollutant_key="PM10",
    pollutant_label="PM$_{10}$",
    year_start=HOURLY_PM10_START_YEAR, year_end=HOURLY_PM10_END_YEAR,
    out_png="PM10_hourly_climatology_harmonics_12panel_stage1_3A.png"
)

plot_monthly_harmonic_climatology_new(
    df_hclim_pm25, pollutant_key="PM25",
    pollutant_label="PM$_{2.5}$",
    year_start=HOURLY_PM25_START_YEAR, year_end=HOURLY_PM25_END_YEAR,
    out_png="PM25_hourly_climatology_harmonics_12panel_stage1_3A.png"
)

Saved: PM10_hourly_climatology_harmonics_12panel_stage1_3A.png
Saved: PM25_hourly_climatology_harmonics_12panel_stage1_3A.png


### Only Phase I threshold

In [166]:
def plot_monthly_harmonic_climatology_new_phase1only(df_clim, pollutant_key, pollutant_label, year_start, year_end, out_png):
    """
    12 panels:
      - ALL years shown (same style)
      - harmonic fit in black computed from multi-year mean (24h+12h)
      - threshold: Phase I only (red, solid)
      - shared y-limits across 12 panels, forced to include Phase I
    """

    fig, axes = plt.subplots(3, 4, figsize=(16, 9), dpi=250, sharex=True, sharey=False)
    axes = axes.flatten()

    hours = np.arange(1, 25)
    color_cycle = plt.rcParams['axes.prop_cycle'].by_key().get('color', None)

    thr1 = PCAA_THR[pollutant_key]["phase1"]

    # First pass: compute global min/max INCLUDING Phase I threshold so it is always visible
    global_min, global_max = np.inf, -np.inf

    for m in range(1, 13):
        subm = df_clim[df_clim["month"] == m]
        if not subm.empty:
            global_min = min(global_min, np.nanmin(subm["mean_conc"].values))
            global_max = max(global_max, np.nanmax(subm["mean_conc"].values))

            mean_all = subm.groupby("hour")["mean_conc"].mean().reindex(hours)
            y_mean = mean_all.values
            if np.isfinite(y_mean).sum() >= 10:
                yhat, _ = fit_diurnal_harmonics(hours, y_mean)
                global_min = min(global_min, np.nanmin(yhat))
                global_max = max(global_max, np.nanmax(yhat))

    # Force include ONLY Phase I threshold (zoom out)
    global_min = min(global_min, thr1)
    global_max = max(global_max, thr1)

    # Add padding
    if np.isfinite(global_min) and np.isfinite(global_max):
        span = global_max - global_min
        pad = 0.08 * span if span > 0 else 5.0
        y_lo = max(0, global_min - pad)
        y_hi = global_max + pad
    else:
        y_lo, y_hi = 0, thr1 * 1.2

    # Second pass: plot
    for i, m in enumerate(range(1, 13)):
        ax = axes[i]
        subm = df_clim[df_clim["month"] == m].copy()

        years = sorted(subm["year"].unique())
        for idx, y in enumerate(years):
            sy = subm[subm["year"] == y].set_index("hour").reindex(hours)
            ax.plot(
                hours,
                sy["mean_conc"].values,
                linewidth=1.0,
                alpha=0.85,
                linestyle="--",
                color=(color_cycle[idx % len(color_cycle)] if color_cycle else None),
            )

        # Harmonic fit on multi-year mean
        mean_all = subm.groupby("hour")["mean_conc"].mean().reindex(hours)
        y_mean = mean_all.values
        if np.isfinite(y_mean).sum() >= 10:
            yhat, _ = fit_diurnal_harmonics(hours, y_mean)
            ax.plot(hours, yhat, color="black", linewidth=1.8, label="Harmonic fit")
        else:
            ax.plot(hours, y_mean, color="black", linewidth=1.8, label="Mean (insufficient)")

        # Threshold: Phase I only
        ax.axhline(thr1, color="red", linewidth=2, alpha=0.95, ls="--")

        ax.set_ylim(y_lo, y_hi)
        ax.set_title(f"Hourly mean {pollutant_label} in {calendar.month_name[m]}", fontsize=10, fontweight="bold")
        ax.set_xlim(1, 24)
        ax.set_xticks(np.arange(1, 25, 2))
        ax.grid(True, alpha=0.3)

        if i % 4 == 0:
            ax.set_ylabel(f"{pollutant_label} [µg/m³]")
        if i >= 8:
            ax.set_xlabel("Hour")

    fig.suptitle(
        f"Hourly climatology by month with diurnal + semidiurnal harmonic fit ({year_start}–{year_end})\n"
        f"Threshold: Phase I (red)",
        fontsize=13, fontweight="bold", y=0.98
    )

    legend_items = [
        Line2D([0],[0], color="black", lw=1.8, label="Harmonic fit (24h+12h)"),
        Line2D([0],[0], color="red",   lw=2, ls="--", label=f"Phase I = {thr1:g} µg/m³"),
        Line2D([0],[0], color="0.3",   lw=1.0, ls="--", label="All years (same weight)"),
    ]
    fig.legend(handles=legend_items, loc="upper left", bbox_to_anchor=(0.01, 0.98), frameon=False, fontsize=9)

    plt.tight_layout(rect=[0, 0, 1, 0.93])
    plt.savefig(out_png, dpi=300, bbox_inches="tight")
    plt.close(fig)
    print(f"Saved: {out_png}")

In [167]:
plot_monthly_harmonic_climatology_new_phase1only(
    df_hclim_pm10, pollutant_key="PM10",
    pollutant_label="PM$_{10}$",
    year_start=HOURLY_PM10_START_YEAR, year_end=HOURLY_PM10_END_YEAR,
    out_png="PM10_hourly_climatology_harmonics_12panel_stage1_3A_phase1only.png"
)

plot_monthly_harmonic_climatology_new_phase1only(
    df_hclim_pm25, pollutant_key="PM25",
    pollutant_label="PM$_{2.5}$",
    year_start=HOURLY_PM25_START_YEAR, year_end=HOURLY_PM25_END_YEAR,
    out_png="PM25_hourly_climatology_harmonics_12panel_stage1_3A_phase1only.png"
)

Saved: PM10_hourly_climatology_harmonics_12panel_stage1_3A_phase1only.png
Saved: PM25_hourly_climatology_harmonics_12panel_stage1_3A_phase1only.png


## 2. Fit computed from last 4 years + print coefficients

In [168]:
def plot_monthly_harmonic_climatology_stage13b_last4only(df_clim, pollutant_key, pollutant_label, out_png, last_n_years=4):
    """
    12 panels:
      - ONLY last 4 years are plotted
      - harmonic fit in black computed from last 4-year mean (24h+12h)
      - thresholds: Phase I (red solid) + Phase II (blue solid)
      - shared y-limits across 12 panels, forced to include thresholds
    """

    years_sorted = sorted(df_clim["year"].unique())
    last_years = years_sorted[-last_n_years:]

    thr1 = PCAA_THR[pollutant_key]["phase1"]
    thr2 = PCAA_THR[pollutant_key]["phase2"]

    fig, axes = plt.subplots(3, 4, figsize=(16, 9), dpi=250, sharex=True, sharey=False)
    axes = axes.flatten()

    hours = np.arange(1, 25)
    global_min, global_max = np.inf, -np.inf

    # First pass: compute y-lims from ONLY last 4 years + fit + thresholds
    for m in range(1, 13):
        subm = df_clim[(df_clim["month"] == m) & (df_clim["year"].isin(last_years))]
        if subm.empty:
            continue

        global_min = min(global_min, np.nanmin(subm["mean_conc"].values))
        global_max = max(global_max, np.nanmax(subm["mean_conc"].values))

        mean_last = subm.groupby("hour")["mean_conc"].mean().reindex(hours)
        y_last = mean_last.values
        if np.isfinite(y_last).sum() >= 10:
            yhat, _ = fit_diurnal_harmonics(hours, y_last)
            global_min = min(global_min, np.nanmin(yhat))
            global_max = max(global_max, np.nanmax(yhat))

    global_min = min(global_min, thr1, thr2)
    global_max = max(global_max, thr1, thr2)

    span = global_max - global_min
    pad = 0.08 * span if span > 0 else 5.0
    y_lo = max(0, global_min - pad)
    y_hi = global_max + pad

    # Plot
    for i, m in enumerate(range(1, 13)):
        ax = axes[i]
        subm = df_clim[(df_clim["month"] == m) & (df_clim["year"].isin(last_years))].copy()

        if subm.empty:
            ax.set_title(f"{calendar.month_name[m]} (no data)")
            ax.set_xlim(1, 24)
            ax.set_ylim(y_lo, y_hi)
            ax.grid(True, alpha=0.3)
            continue

        # last 4 years only
        for y in last_years:
            sy = subm[subm["year"] == y].set_index("hour").reindex(hours)
            ax.plot(hours, sy["mean_conc"].values, linewidth=1.2, alpha=0.95, linestyle="--", label=str(y))

        # harmonic fit from last 4 years
        mean_last = subm.groupby("hour")["mean_conc"].mean().reindex(hours)
        y_last = mean_last.values
        if np.isfinite(y_last).sum() >= 10:
            yhat, _ = fit_diurnal_harmonics(hours, y_last)
            ax.plot(hours, yhat, color="black", linewidth=2.0, label="Harmonic fit (last 4 yrs)")
        else:
            ax.plot(hours, y_last, color="black", linewidth=2.0, label="Mean (insufficient)")

        # thresholds
        ax.axhline(thr1, color="red",  linewidth=2, alpha=0.95, ls="--")
        ax.axhline(thr2, color="blue", linewidth=2, alpha=0.95, ls="--")

        ax.set_ylim(y_lo, y_hi)
        ax.set_title(f"Hourly mean {pollutant_label} in {calendar.month_name[m]}", fontsize=10, fontweight="bold")
        ax.set_xlim(1, 24)
        ax.set_xticks(np.arange(1, 25, 2))
        ax.grid(True, alpha=0.3)

        if i % 4 == 0:
            ax.set_ylabel(f"{pollutant_label} [µg/m³]")
        if i >= 8:
            ax.set_xlabel("Hour")

    fig.suptitle(
        f"Hourly climatology by month (last {last_n_years} years: {last_years[0]}–{last_years[-1]})\n"
        f"Diurnal + semidiurnal harmonic fit; Thresholds: Phase I (red), Phase II (blue)",
        fontsize=13, fontweight="bold", y=0.98
    )

    # Legend: only last 4 + fit + thresholds
    handles, labels = axes[0].get_legend_handles_labels()
    extra = [
        Line2D([0],[0], color="red",  lw=2, label=f"Phase I = {thr1:g} µg/m³"),
        Line2D([0],[0], color="blue", lw=2, label=f"Phase II = {thr2:g} µg/m³"),
    ]
    fig.legend(handles + extra, labels + [e.get_label() for e in extra],
               loc="upper left", bbox_to_anchor=(0.01, 0.98), frameon=False, fontsize=9, ncol=2)

    plt.tight_layout(rect=[0, 0, 1, 0.93])
    plt.savefig(out_png, dpi=300, bbox_inches="tight")
    plt.close(fig)
    print(f"Saved: {out_png}")
    return last_years


In [169]:
def summarize_diurnal_harmonics_last4(df_clim, label, last_n_years=4):

    years_sorted = sorted(df_clim["year"].unique())
    last_years = years_sorted[-last_n_years:]

    hours = np.arange(1, 25)
    rows = []
    for m in range(1, 13):
        subm = df_clim[(df_clim["month"] == m) & (df_clim["year"].isin(last_years))]
        y_mean = subm.groupby("hour")["mean_conc"].mean().reindex(hours)
        y = y_mean.values
        if np.isfinite(y).sum() < 10:
            continue
        _, beta = fit_diurnal_harmonics(hours, y)
        a0, a1, b1, a2, b2 = beta
        rows.append({
            "month": calendar.month_abbr[m],
            "years_used": f"{last_years[0]}–{last_years[-1]}",
            "a0": a0,
            "a1": a1,
            "b1": b1,
            "a2": a2,
            "b2": b2,
            "amp24h": np.sqrt(a1**2 + b1**2),
            "amp12h": np.sqrt(a2**2 + b2**2)
        })

    if not rows:
        print(f"No valid monthly means for {label} (last {last_n_years} years).")
        return None

    df_out = pd.DataFrame(rows).round(3)
    display(HTML(f"<h3 style='margin:4px 0;'>{label} (fit from last {last_n_years} years)</h3>"))
    display(df_out)
    return df_out

In [170]:
# PM10
last_years_pm10 = plot_monthly_harmonic_climatology_stage13b_last4only(
    df_hclim_pm10, pollutant_key="PM10",
    pollutant_label="PM$_{10}$",
    out_png="PM10_hourly_climatology_harmonics_12panel_stage1_3B_last4only.png",
    last_n_years=4
)
pm10_harmonics_last4 = summarize_diurnal_harmonics_last4(
    df_hclim_pm10,
    "Diurnal + semidiurnal coefficients for PM10",
    last_n_years=4
)

# PM2.5
last_years_pm25 = plot_monthly_harmonic_climatology_stage13b_last4only(
    df_hclim_pm25, pollutant_key="PM25",
    pollutant_label="PM$_{2.5}$",
    out_png="PM25_hourly_climatology_harmonics_12panel_stage1_3B_last4only.png",
    last_n_years=4
)
pm25_harmonics_last4 = summarize_diurnal_harmonics_last4(
    df_hclim_pm25,
    "Diurnal + semidiurnal coefficients for PM2.5",
    last_n_years=4
)

Saved: PM10_hourly_climatology_harmonics_12panel_stage1_3B_last4only.png


,month,years_used,a0,a1,b1,a2,b2,amp24h,amp12h
0,Jan,2020–2023,47.10,-4.78,-2.61,0.10,-8.49,5.44,8.49
1,Feb,2020–2023,48.88,-6.12,-5.31,-0.50,-8.95,8.10,8.96
2,Mar,2020–2023,53.62,-6.03,-7.12,-4.17,-8.65,9.33,9.61
3,Apr,2020–2023,47.64,-6.41,-4.07,-5.40,-6.09,7.59,8.14
4,May,2020–2023,40.80,-5.90,-2.55,-3.90,-5.49,6.43,6.74
5,Jun,2020–2023,29.93,-4.27,-3.61,-2.48,-3.76,5.59,4.51
6,Jul,2020–2023,27.76,-7.97,-2.38,-0.30,-2.50,8.32,2.52
7,Aug,2020–2023,23.95,-7.45,-2.15,-0.18,-1.99,7.75,2.00
8,Sep,2020–2023,22.93,-6.01,-2.65,-0.93,-2.06,6.57,2.25
9,Oct,2020–2023,32.55,-7.27,-2.47,-1.92,-5.11,7.68,5.46


Saved: PM25_hourly_climatology_harmonics_12panel_stage1_3B_last4only.png


,month,years_used,a0,a1,b1,a2,b2,amp24h,amp12h
0,Jan,2020–2023,21.03,-3.23,2.63,1.38,-1.34,4.16,1.92
1,Feb,2020–2023,19.98,-3.29,1.08,1.15,-1.39,3.46,1.80
2,Mar,2020–2023,22.58,-3.65,1.43,0.43,-1.50,3.92,1.56
3,Apr,2020–2023,24.58,-3.29,1.23,-0.40,-1.11,3.51,1.18
4,May,2020–2023,23.07,-3.83,1.23,0.34,-1.31,4.03,1.35
5,Jun,2020–2023,14.93,-3.08,0.24,1.08,-0.57,3.09,1.22
6,Jul,2020–2023,15.21,-5.04,-0.39,1.92,-0.45,5.06,1.98
7,Aug,2020–2023,13.28,-4.53,-0.67,1.68,-0.09,4.58,1.69
8,Sep,2020–2023,12.81,-3.48,-0.66,1.22,0.10,3.54,1.23
9,Oct,2020–2023,16.74,-4.40,0.34,0.84,-0.85,4.42,1.19


### Only Phase I threshold

In [171]:
def plot_monthly_harmonic_climatology_stage13b_last4only_phase1(df_clim, pollutant_key, pollutant_label, out_png, last_n_years=4):
    """
    12 panels:
      - ONLY last 4 years are plotted
      - harmonic fit in black computed from last 4-year mean (24h+12h)
      - threshold: Phase I only (red solid)
      - shared y-limits across 12 panels, forced to include Phase I
    """

    years_sorted = sorted(df_clim["year"].unique())
    last_years = years_sorted[-last_n_years:]

    thr1 = PCAA_THR[pollutant_key]["phase1"]

    fig, axes = plt.subplots(3, 4, figsize=(16, 9), dpi=250, sharex=True, sharey=False)
    axes = axes.flatten()

    hours = np.arange(1, 25)
    global_min, global_max = np.inf, -np.inf

    # First pass: compute y-lims from ONLY last 4 years + fit + Phase I threshold
    for m in range(1, 13):
        subm = df_clim[(df_clim["month"] == m) & (df_clim["year"].isin(last_years))]
        if subm.empty:
            continue

        global_min = min(global_min, np.nanmin(subm["mean_conc"].values))
        global_max = max(global_max, np.nanmax(subm["mean_conc"].values))

        mean_last = subm.groupby("hour")["mean_conc"].mean().reindex(hours)
        y_last = mean_last.values
        if np.isfinite(y_last).sum() >= 10:
            yhat, _ = fit_diurnal_harmonics(hours, y_last)
            global_min = min(global_min, np.nanmin(yhat))
            global_max = max(global_max, np.nanmax(yhat))

    global_min = min(global_min, thr1)
    global_max = max(global_max, thr1)

    span = global_max - global_min
    pad = 0.08 * span if span > 0 else 5.0
    y_lo = max(0, global_min - pad)
    y_hi = global_max + pad

    # Plot
    for i, m in enumerate(range(1, 13)):
        ax = axes[i]
        subm = df_clim[(df_clim["month"] == m) & (df_clim["year"].isin(last_years))].copy()

        if subm.empty:
            ax.set_title(f"{calendar.month_name[m]} (no data)")
            ax.set_xlim(1, 24)
            ax.set_ylim(y_lo, y_hi)
            ax.grid(True, alpha=0.3)
            continue

        # last 4 years only
        for y in last_years:
            sy = subm[subm["year"] == y].set_index("hour").reindex(hours)
            ax.plot(hours, sy["mean_conc"].values, linewidth=1.2, alpha=0.95, linestyle="--", label=str(y))

        # harmonic fit from last 4 years
        mean_last = subm.groupby("hour")["mean_conc"].mean().reindex(hours)
        y_last = mean_last.values
        if np.isfinite(y_last).sum() >= 10:
            yhat, _ = fit_diurnal_harmonics(hours, y_last)
            ax.plot(hours, yhat, color="black", linewidth=2.0, label="Harmonic fit (last 4 yrs)")
        else:
            ax.plot(hours, y_last, color="black", linewidth=2.0, label="Mean (insufficient)")

        # threshold: Phase I only
        ax.axhline(thr1, color="red", linewidth=2, alpha=0.95, ls="--")

        ax.set_ylim(y_lo, y_hi)
        ax.set_title(f"Hourly mean {pollutant_label} in {calendar.month_name[m]}", fontsize=10, fontweight="bold")
        ax.set_xlim(1, 24)
        ax.set_xticks(np.arange(1, 25, 2))
        ax.grid(True, alpha=0.3)

        if i % 4 == 0:
            ax.set_ylabel(f"{pollutant_label} [µg/m³]")
        if i >= 8:
            ax.set_xlabel("Hour")

    fig.suptitle(
        f"Hourly climatology by month (last {last_n_years} years: {last_years[0]}–{last_years[-1]})\n"
        f"Diurnal + semidiurnal harmonic fit; Threshold: Phase I (red)",
        fontsize=13, fontweight="bold", y=0.98
    )

    # Legend: last 4 + fit + Phase I
    handles, labels = axes[0].get_legend_handles_labels()
    extra = [Line2D([0],[0], color="red", lw=2, ls="--", label=f"Phase I = {thr1:g} µg/m³")]
    fig.legend(handles + extra, labels + [e.get_label() for e in extra],
               loc="upper left", bbox_to_anchor=(0.01, 0.98), frameon=False, fontsize=9, ncol=2)

    plt.tight_layout(rect=[0, 0, 1, 0.93])
    plt.savefig(out_png, dpi=300, bbox_inches="tight")
    plt.close(fig)
    print(f"Saved: {out_png}")
    return last_years

In [172]:
last_years_pm10 = plot_monthly_harmonic_climatology_stage13b_last4only_phase1(
    df_hclim_pm10, pollutant_key="PM10",
    pollutant_label="PM$_{10}$",
    out_png="PM10_hourly_climatology_harmonics_12panel_stage1_3B_last4_phase1only.png",
    last_n_years=4
)

last_years_pm25 = plot_monthly_harmonic_climatology_stage13b_last4only_phase1(
    df_hclim_pm25, pollutant_key="PM25",
    pollutant_label="PM$_{2.5}$",
    out_png="PM25_hourly_climatology_harmonics_12panel_stage1_3B_last4_phase1only.png",
    last_n_years=4
)

Saved: PM10_hourly_climatology_harmonics_12panel_stage1_3B_last4_phase1only.png
Saved: PM25_hourly_climatology_harmonics_12panel_stage1_3B_last4_phase1only.png


## 3. PCAA Phase I composites (MA24 >= Phase I threshold)

In [173]:
def compute_ma24(df_hourly: pd.DataFrame, value_col: str, min_hours: int = 18) -> pd.DataFrame:
    """
    df_hourly must have:
      - timestamp (datetime64)
      - value_col (float)
    Returns df with an added column: 'ma24'
    Rule: MA24 computed only if at least min_hours non-missing within the 24h window.
    """
    d = df_hourly.copy()
    d = d.sort_values("timestamp").set_index("timestamp")

    s = d[value_col].astype(float)

    # Rolling 24h mean and rolling count of valid hours
    ma = s.rolling(window=24, min_periods=min_hours).mean()
    d["ma24"] = ma

    return d.reset_index()

In [174]:
def daily_max_ma24(df_ma24: pd.DataFrame, pollutant_key: str) -> pd.DataFrame:
    """
    df_ma24 must have:
      - timestamp
      - ma24
    Output per day:
      date, timestamp (time of max MA24), daily_max_ma24, hour_of_max, ge_phase1, ge_phase2
    """
    d = df_ma24.copy()
    d["timestamp"] = pd.to_datetime(d["timestamp"])
    d["date"] = d["timestamp"].dt.floor("D")

    # pick timestamp of max MA24 each day (ignore days with all-NaN)
    tmp = d.dropna(subset=["ma24"]).copy()
    if tmp.empty:
        return pd.DataFrame(columns=["date","timestamp","daily_max_ma24","hour_of_max","ge_phase1","ge_phase2"])

    idx = tmp.groupby("date")["ma24"].idxmax()
    out = tmp.loc[idx, ["date", "timestamp", "ma24"]].copy()
    out = out.rename(columns={"ma24": "daily_max_ma24"})
    out["hour_of_max"] = out["timestamp"].dt.hour

    thr1 = PCAA_THR[pollutant_key]["phase1"]
    thr2 = PCAA_THR[pollutant_key]["phase2"]
    out["ge_phase1"] = out["daily_max_ma24"] >= thr1
    out["ge_phase2"] = out["daily_max_ma24"] >= thr2

    out = out.sort_values("date").reset_index(drop=True)
    return out

In [175]:
def build_events_by_month_from_pcaa(daily_df: pd.DataFrame,
                                    flag_col: str,
                                    start: str = PCAA_START,
                                    end: str = PCAA_END):
    """
    Returns dict month->DatetimeIndex of event days (normalized).
    """
    d = daily_df.copy()
    d["date"] = pd.to_datetime(d["date"])
    d = d[(d["date"] >= pd.to_datetime(start)) & (d["date"] <= pd.to_datetime(end))]

    ev = d[d[flag_col] == True].copy()

    out = {m: pd.DatetimeIndex([]) for m in range(1, 13)}
    for m, g in ev.groupby(ev["date"].dt.month):
        out[int(m)] = pd.DatetimeIndex(sorted(g["date"].dt.normalize().unique()))

    return out, ev

In [176]:
# --- Hourly "worst-station" series (already defined in section 7)
pm10_hourly = build_hourly_series("PM10", YEARS_PM10)  # expects columns: timestamp, area_max
pm25_hourly = build_hourly_series("PM25", YEARS_PM25)

# --- MA24
pm10_ma24 = compute_ma24(pm10_hourly, value_col="area_max", min_hours=18)
pm25_ma24 = compute_ma24(pm25_hourly, value_col="area_max", min_hours=18)

# --- Daily max of MA24 + event flags
pm10_daily_ma24 = daily_max_ma24(pm10_ma24, pollutant_key="PM10")
pm25_daily_ma24 = daily_max_ma24(pm25_ma24, pollutant_key="PM25")

# --- Save CSVs for traceability
pm10_daily_ma24.to_csv("PM10_PCAA_MA24_dailymax.csv", index=False)
pm25_daily_ma24.to_csv("PM25_PCAA_MA24_dailymax.csv", index=False)

print("Saved: PM10_PCAA_MA24_dailymax.csv")
print("Saved: PM25_PCAA_MA24_dailymax.csv")


C:\Users\DELL\AppData\Local\Temp\ipykernel_1672\4039281186.py:66: FutureWarning: 'H' is deprecated and will be removed in a future version. Please use 'h' instead of 'H'.
  out["timestamp"] = out["date_adj"] + pd.to_timedelta(out["hour_adj"], unit="H")
C:\Users\DELL\AppData\Local\Temp\ipykernel_1672\4039281186.py:66: FutureWarning: 'H' is deprecated and will be removed in a future version. Please use 'h' instead of 'H'.
  out["timestamp"] = out["date_adj"] + pd.to_timedelta(out["hour_adj"], unit="H")
C:\Users\DELL\AppData\Local\Temp\ipykernel_1672\4039281186.py:66: FutureWarning: 'H' is deprecated and will be removed in a future version. Please use 'h' instead of 'H'.
  out["timestamp"] = out["date_adj"] + pd.to_timedelta(out["hour_adj"], unit="H")
C:\Users\DELL\AppData\Local\Temp\ipykernel_1672\4039281186.py:66: FutureWarning: 'H' is deprecated and will be removed in a future version. Please use 'h' instead of 'H'.
  out["timestamp"] = out["date_adj"] + pd.to_timedelta(out["hour_adj"]

Saved: PM10_PCAA_MA24_dailymax.csv
Saved: PM25_PCAA_MA24_dailymax.csv


In [181]:
events_pcaa1_pm10, ev_pcaa1_pm10 = build_events_by_month_from_pcaa(pm10_daily_ma24, "ge_phase1")
events_pcaa1_pm25, ev_pcaa1_pm25 = build_events_by_month_from_pcaa(pm25_daily_ma24, "ge_phase1")

print("PCAA Phase I counts (2012–2023):")
print("PM10 total:", len(ev_pcaa1_pm10), "| by month:", {m: len(events_pcaa1_pm10[m]) for m in range(1,13)})
print("PM2.5 total:", len(ev_pcaa1_pm25), "| by month:", {m: len(events_pcaa1_pm25[m]) for m in range(1,13)})

PCAA Phase I counts (2012–2023):
PM10 total: 28 | by month: {1: 13, 2: 2, 3: 4, 4: 2, 5: 0, 6: 0, 7: 0, 8: 0, 9: 0, 10: 0, 11: 0, 12: 7}
PM2.5 total: 49 | by month: {1: 18, 2: 2, 3: 1, 4: 5, 5: 7, 6: 0, 7: 0, 8: 0, 9: 0, 10: 0, 11: 0, 12: 16}


In [184]:
plot_monthly_multipanel(
    pol="PM10 (PCAA Phase I; MA24 worst-station)",
    events_by_month=events_pcaa1_pm10,
    Hprime=Hprime500, U=U500, V=V500,
    outname="Z500_monthly_PCAA_phase1_PM10.png",
    label="Phase I exceedance days",
    threshold_note="PM10 ≥ 214 µg/m³ (24h MA)",
    period="2012–2023"
)

plot_monthly_multipanel(
    pol="PM2.5 (PCAA Phase I; MA24 worst-station)",
    events_by_month=events_pcaa1_pm25,
    Hprime=Hprime500, U=U500, V=V500,
    outname="Z500_monthly_PCAA_phase1_PM25.png",
    label="Phase I exceedance days",
    threshold_note="PM2.5 ≥ 97.40 µg/m³ (24h MA)",
    period="2012–2023"
)

C:\Users\DELL\AppData\Local\Temp\ipykernel_1672\2386312794.py:159: UserWarning: This figure includes Axes that are not compatible with tight_layout, so results might be incorrect.
  plt.tight_layout(rect=[0, 0, 0.9, 0.95])


Saved: Z500_monthly_PCAA_phase1_PM10.png


C:\Users\DELL\AppData\Local\Temp\ipykernel_1672\2386312794.py:159: UserWarning: This figure includes Axes that are not compatible with tight_layout, so results might be incorrect.
  plt.tight_layout(rect=[0, 0, 0.9, 0.95])


Saved: Z500_monthly_PCAA_phase1_PM25.png


## 4. PCAA Phase II composites (MA24 >= Phase II threshold)

In [185]:
events_pcaa2_pm10, ev_pcaa2_pm10 = build_events_by_month_from_pcaa(pm10_daily_ma24, "ge_phase2")
events_pcaa2_pm25, ev_pcaa2_pm25 = build_events_by_month_from_pcaa(pm25_daily_ma24, "ge_phase2")

print("PCAA Phase II counts (2012–2023):")
print("PM10 total:", len(ev_pcaa2_pm10), "| by month:", {m: len(events_pcaa2_pm10[m]) for m in range(1,13)})
print("PM2.5 total:", len(ev_pcaa2_pm25), "| by month:", {m: len(events_pcaa2_pm25[m]) for m in range(1,13)})

PCAA Phase II counts (2012–2023):
PM10 total: 0 | by month: {1: 0, 2: 0, 3: 0, 4: 0, 5: 0, 6: 0, 7: 0, 8: 0, 9: 0, 10: 0, 11: 0, 12: 0}
PM2.5 total: 8 | by month: {1: 3, 2: 2, 3: 0, 4: 0, 5: 0, 6: 0, 7: 0, 8: 0, 9: 0, 10: 0, 11: 0, 12: 3}


In [187]:
plot_monthly_multipanel(
    pol="PM2.5 (PCAA Phase II; MA24 worst-station)",
    events_by_month=events_pcaa2_pm25,
    Hprime=Hprime500, U=U500, V=V500,
    outname="Z500_monthly_PCAA_phase2_PM25.png",
    label="Phase II exceedance days",
    threshold_note="PM2.5 ≥ 150.40 µg/m³ (24h MA)",
    period="2012–2023"
)

C:\Users\DELL\AppData\Local\Temp\ipykernel_1672\2386312794.py:159: UserWarning: This figure includes Axes that are not compatible with tight_layout, so results might be incorrect.
  plt.tight_layout(rect=[0, 0, 0.9, 0.95])


Saved: Z500_monthly_PCAA_phase2_PM25.png
